In [1]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("C://Users/Emrehan/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         3 non-null      object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


In [2]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import concurrent.futures

# Geckodriver'ın yolu
driver_path = "C:\\Users\\EMREHAN\\PycharmProjects\\geckodriver.exe"

def fetch_price(product_url):
    """
    Belirtilen ürün URL'sinden fiyat bilgisini çekip döndürür.
    Her çağrıda yeni bir Selenium driver oluşturulur.
    """
    price = "N/A"  # Varsayılan fiyat değeri
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    # (İsteğe bağlı) User-Agent ayarı yapılabilir:
    # options.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...")
    service = Service(driver_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        driver.get(product_url)

        # 1. Yöntem: Hiyerarşik yapıdaki fiyat bilgisini çekmeye çalışalım.
        try:
            price_container = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    "span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay"
                ))
            )
            # Fiyat bileşenlerini alalım
            whole = price_container.find_element(By.CSS_SELECTOR, "span.a-price-whole").text.strip()
            decimal = price_container.find_element(By.CSS_SELECTOR, "span.a-price-decimal").text.strip()
            try:
                fraction = price_container.find_element(By.CSS_SELECTOR, "span.a-price-fraction").text.strip()
                price = f"{whole}{decimal}{fraction}"
            except Exception:
                price = f"{whole}{decimal}"
            symbol = price_container.find_element(By.CSS_SELECTOR, "span.a-price-symbol").text.strip()
            price = f"{price} {symbol}"

        except Exception as hierarchical_exception:
            # 2. Yöntem: 'span.a-offscreen' elementini bekleyelim.
            try:
                price_element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-offscreen"))
                )
                price = price_element.text.strip()
            except Exception as e:
                # 3. Yöntem: 'a-price-whole' ve 'a-price-fraction' kombinasyonunu deneyelim.
                try:
                    whole_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-price-whole"))
                    )
                    fraction_element = driver.find_element(By.CSS_SELECTOR, "span.a-price-fraction")
                    price = f"{whole_element.text.strip()},{fraction_element.text.strip()} TL"
                except Exception as e2:
                    # 4. Yöntem: Son çare olarak BeautifulSoup ile sayfa kaynak kodunu kontrol edelim.
                    time.sleep(2)  # Ekstra bekleme
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    price_tag = soup.find("span", class_="a-offscreen")
                    if price_tag:
                        price = price_tag.text.strip()
                    else:
                        whole_tag = soup.find("span", class_="a-price-whole")
                        fraction_tag = soup.find("span", class_="a-price-fraction")
                        if whole_tag and fraction_tag:
                            price = f"{whole_tag.text.strip()},{fraction_tag.text.strip()} TL"
                        else:
                            price = "N/A"

    except Exception as main_exception:
        price = "N/A"
    finally:
        driver.quit()

    return price

if __name__ == "__main__":
    # CSV dosyasını oku
    csv_file_path = "C://Users/EMREHAN/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv"
    df = pd.read_csv(csv_file_path)

    # Eğer "Price" kolonu yoksa, oluştur
    if "Price" not in df.columns:
        df["Price"] = "N/A"

    # Ürün URL'lerini listeye alalım
    urls = df["Link"].tolist()

    # Paralel işlemler için ThreadPoolExecutor kullanıyoruz.
    # max_workers: aynı anda kaç tarayıcı örneği çalıştırılacağını belirler.
    max_workers = 5
    prices = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Her URL için fetch_price fonksiyonunu çalıştırıyoruz.
        future_to_url = {executor.submit(fetch_price, url): url for url in urls}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                price = future.result()
            except Exception as exc:
                print(f"{url} için hata oluştu: {exc}")
                price = "N/A"
            prices.append(price)
            print(f"URL: {url} => Fiyat: {price}")

    # Elde edilen fiyatları DataFrame'e kaydedelim.
    df["Price"] = prices

    # Güncellenmiş CSV'yi kaydedelim
    output_csv = "C://Users/EMREHAN/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv"
    df.to_csv(output_csv, index=False)

    print("Fiyat bilgileri güncellendi ve CSV'ye kaydedildi.")


URL: https://www.amazon.com.tr/ASUS-Vivobook-15-X1504ZA-NJ566-Bilgisayar/dp/B08HLQTYXW/ref=sr_1_4?dib=eyJ2IjoiMSJ9.cAStGyXY4Ttlop_Td9PZXiT8XUwEgMm5EFh6-h95Y1hWj9uUUBoaD8md3A4UjnjuIagUE3vJx3KnUi-JvH8cKk0HTEGAX8kO0Bhtc8QUjYa2l6_zy0ln6kP2qI_6Kxoh-nDmgMQKQl-tP6Q3U9i6Xdo8nrDd85-9hZEJrFyfJIHDVKHvBuVDoc8__xz73XZ515r2uBFMSt5cRM1v22ysRKWFcdViGJQCj0yZA6WUJqAwlS7Qbx81kqBIQYsncmEogGLZXwAk8vb032RdTv_eCvYz9uDT9D7To-w7UHkP2VrMV276jD3bNLuT6iBIwgFyMQ8yLp_hU-b_xy0aYApkCHejwbw8mlO-w_2rrCVVzck.Oyo7D-r7aZUqHTPMtdljGI-kKT6WVuVoCeztVUvTInw&dib_tag=se&qid=1738967976&s=computers&sr=1-4 => Fiyat: 15.49900 TL
URL: https://www.amazon.com.tr/Lenovo-Bilgisayar-i5-12450H-Integrated-83ER000WTR/dp/B0CK84VL9X/ref=sr_1_1?dib=eyJ2IjoiMSJ9.cAStGyXY4Ttlop_Td9PZXiT8XUwEgMm5EFh6-h95Y1hWj9uUUBoaD8md3A4UjnjuIagUE3vJx3KnUi-JvH8cKk0HTEGAX8kO0Bhtc8QUjYa2l6_zy0ln6kP2qI_6Kxoh-nDmgMQKQl-tP6Q3U9i6Xdo8nrDd85-9hZEJrFyfJIHDVKHvBuVDoc8__xz73XZ515r2uBFMSt5cRM1v22ysRKWFcdViGJQCj0yZA6WUJqAwlS7Qbx81kqBIQYsncmEogGLZXwAk8vb032RdTv_eCvYz9uDT9D7

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         186 non-null    object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


In [5]:
df[df["Price"]==""]

,Product Name,Price,Image URL,Link,ASIN,Review
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...,B0DCYYFMYC,Ürünü bilgisayar öğretmeni arkadaşımın tavsiye...
13,"MSI THIN 15 B12VE-1625XTR Dizüstü Bilgisayar, ...",,https://m.media-amazon.com/images/I/71Fr-EEsp6...,https://www.amazon.com.tr/MSI-B12VE-1625XTR-Di...,B0CZ3LWDVR,Cihazı teknik özelliklerine bakarak aldım ve b...
15,Acer Aspire 3 A315-510P-38X0 Dizüstü Bilgisaya...,,https://m.media-amazon.com/images/I/61wQJ72EP3...,https://www.amazon.com.tr/Acer-A315-510P-38X0-...,B0C1K8T32W,Öncelikle Windows kurulumundan bahsedeyim. Baş...
27,HP 250 G9 9M3G5AT02 i5-1235U 16GB 512SSD 15.6 ...,,https://m.media-amazon.com/images/I/71OsrD+1wK...,https://www.amazon.com.tr/HP-9M3G5AT02-i5-1235...,B0D3TLHPJV,Ürünün iade süreci içerisinde fiyatı düşmesine...
32,MSI MODERN 14 C12MO-1039XTR Dizüstü Bilgisayar...,,https://m.media-amazon.com/images/I/71czorPUNz...,https://www.amazon.com.tr/MSI-C12MO-1039XTR-Di...,B0CYLKNFHT,Kurdum. Kullanmaya başladım. Memnunum.
47,"Huawei Matebook D16 Dizüstü Bilgisayar, 16"" FH...",,https://m.media-amazon.com/images/I/7181PwCFHa...,https://www.amazon.com.tr/Matebook-Diz%C3%BCst...,B0CSDQQRWQ,Çevremde bana bilgisayar lazım hocam diyen ins...
48,Lenovo LOQ i5-12450HX 16GB RAM 512GB SSD 15.6 ...,,https://m.media-amazon.com/images/I/81WlQ4KI9I...,https://www.amazon.com.tr/Lenovo-i5-12450HX-Fr...,B0D1Y35Z9V,Ürünü 3 gün önce yazdım biraz kullandıktan son...
85,MSI SWORD 16 HX B14VFKG-061XTR Dizüstü Bilgisa...,,https://m.media-amazon.com/images/I/51mDrkWLOH...,https://www.amazon.com.tr/MSI-B14VFKG-061XTR-D...,B0CXDTZDL3,Laptopu 4 aydır kullanıyorum ve gayet performa...
88,Asus Vivobook S 15 M5506UA-MA043 AMD Ryzen 9 8...,,https://m.media-amazon.com/images/I/718YHIjvKw...,https://www.amazon.com.tr/Vivobook-M5506UA-MA0...,B0DJR2Z498,Ürün çok iyi. Windows'u mutlaka rufus ile kuru...
96,Monster Tulpar T7 V20.6.2 Intel Core i7 13700H...,,https://m.media-amazon.com/images/I/61sMpEKkId...,https://www.amazon.com.tr/Monster-Tulpar-T7-V2...,B0C7CMJK32,ürünün kalitesi falan her şeyi süper gönül rah...


In [6]:
df["Review"][88]

"Ürün çok iyi. Windows'u mutlaka rufus ile kurun yoksa windows tpm 2.0 ı görmüyor. | TPM 2.0 modülü çalışmıyor, Pil belirtilenden hızlı biten, klavyesi olması gerekenden çok daha kötü, ayrıca Nethouse isimli firma iade politikalarına kesinlikle uyum sağlamıyor. Ürünü üçüncü satıcıdan kesinlikle almayın."

In [47]:
print(df["Link"][185])

https://www.amazon.com.tr/HP-9M3G7AT02-FreeDOS-Ta%C5%9F%C4%B1nabilir-Bilgisayar-CNT003/dp/B0D92N5QWS/ref=sr_1_359?dib=eyJ2IjoiMSJ9.S1JcaV1XK1ub2CL8xGkxBjEyGYockkGnuzYvY7VzOXX5pwn50YTZvQx7PRQLOY4lybR5YwTWXoj_t5lI6b1-lyhVc-qxKmc54qn4UYC6AvxmAZQ-Apeoy2uzX5JowKM0jKfdOLMZBlp2wCyU4ENhgTwhbg0_ogtUH5awzjyRhJRM8oPcnJAtFSA-vNn6u5i71l-q4y9panCPP3V9gyIYbwUnccAMMGNbdhoqssNc7iOhAhAm7x2gmvp6T_h5vH_9CPSS4S4xWWNwhsEi4nqPPQ8dfnK_w8b1Pqe3HaKlzPOQ_tkBqfD7txzgHMa1yVvIUzafVfr7RYe1E2T9_wPl3y0SQJQw9wYemkTmoL5zxgg.KgJMNASpw_07-s39j8RzwHVdkRVxWqMXbqRiDb8W4zE&dib_tag=se&qid=1738968207&s=computers&sr=1-359


In [42]:
df["Price"][170] = "10.30000 TL"

C:\Users\EMREHAN\AppData\Local\Temp\ipykernel_25364\396236172.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Price"][170] = "10.30000 TL"


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         186 non-null    object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


In [49]:
df[df["Price"]==""]

,Product Name,Price,Image URL,Link,ASIN,Review
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...,B0DCYYFMYC,Ürünü bilgisayar öğretmeni arkadaşımın tavsiye...
15,Acer Aspire 3 A315-510P-38X0 Dizüstü Bilgisaya...,,https://m.media-amazon.com/images/I/61wQJ72EP3...,https://www.amazon.com.tr/Acer-A315-510P-38X0-...,B0C1K8T32W,Öncelikle Windows kurulumundan bahsedeyim. Baş...
27,HP 250 G9 9M3G5AT02 i5-1235U 16GB 512SSD 15.6 ...,,https://m.media-amazon.com/images/I/71OsrD+1wK...,https://www.amazon.com.tr/HP-9M3G5AT02-i5-1235...,B0D3TLHPJV,Ürünün iade süreci içerisinde fiyatı düşmesine...
96,Monster Tulpar T7 V20.6.2 Intel Core i7 13700H...,,https://m.media-amazon.com/images/I/61sMpEKkId...,https://www.amazon.com.tr/Monster-Tulpar-T7-V2...,B0C7CMJK32,ürünün kalitesi falan her şeyi süper gönül rah...
101,Lenovo Nb E16 Thınkpad 21JT0017TX RYZEN7-7730U...,,https://m.media-amazon.com/images/I/61LooBApkd...,https://www.amazon.com.tr/Lenovo-Th%C4%B1nkpad...,B0DBM6N3FT,"Ürünü, Ubuntu kurmak için seçtim ve aldım. Gay..."
110,Dell Vostro 3520 N1614PVNB3520U13 i3-1215U 32G...,,https://m.media-amazon.com/images/I/71nbaKV1BN...,https://www.amazon.com.tr/Dell-N1614PVNB3520U1...,B0CZY27KN1,Evde ofis programları kullanmak için aldım. Ço...
122,Monster Abra A5 V20.4 Intel Core i5 12450H 16 ...,,https://m.media-amazon.com/images/I/61tOfogRfX...,https://www.amazon.com.tr/Monster-Abra-A5-V20-...,B0CHTGBBHT,Paketleme güzeldi. hızlı ve akıcı bir pc oyunl...
123,Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...,,https://m.media-amazon.com/images/I/61wH5UlX-M...,https://www.amazon.com.tr/Lenovo-IdeaPad-i5-12...,B0CZRHHRPH,Bu fiyata alabileceğini daha iyi bir notebook ...
134,Acer Nitro V 15 Intel Core i5 13420H 32GB DDR5...,,https://m.media-amazon.com/images/I/616TfZ1AJL...,https://www.amazon.com.tr/Nitro-Intel-13420H-4...,B0DJVWN7WP,"1 ay dolmadan arızalandı, iade talep ettim 10 ..."
137,Dell Vostro 3520 i5-1235U 8GB 256GB SSD 15.6 F...,,https://m.media-amazon.com/images/I/81cPb73-Fh...,https://www.amazon.com.tr/Dell-Vostro-i5-1235U...,B0C8TPB45B,"ürün harika, 120hz ekranı olması yeter. ubuntu..."


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("C://Users/Emrehan/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         154 non-null    object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


In [4]:
def format_price(price):
    if isinstance(price, str) and "TL" in price:
        price = price.replace(" TL", "")  # " TL" kısmını kaldır
        try:
            price = float(price)  # Sayıya çevir
            return f"{price:.3f} TL"  # Noktadan sonra 3 basamak bırak ve TL ekle
        except ValueError:
            return None
    return None

df["Price"] = df["Price"].apply(format_price)

In [5]:
df["Price"].head(10)

0    15.499 TL
1    14.999 TL
2    11.659 TL
3    10.999 TL
4         None
5    45.999 TL
6    46.897 TL
7    40.399 TL
8    10.899 TL
9    84.204 TL
Name: Price, dtype: object

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         165 non-null    object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


In [8]:
import pandas as pd
import numpy as np

missing_prices = df[df["Price"].isna()][["Product Name", "Link"]]

# Manuel olarak fiyatları gir
for index, row in missing_prices.iterrows():
    print(f"\nÜrün: {row['Product Name']}")
    print(f"Link: {row['Link']}")
    new_price = input("Bu ürünün fiyatını gir (örn: 15.499 TL): ")
    
    # Fiyatı DataFrame'e ekle
    df.at[index, "Price"] = new_price


Ürün: Lenovo Nb E16 Thınkpad 21JT0017TX RYZEN7-7730U 16GB 512SSD O/b 16 Dos Notebook
Link: https://www.amazon.com.tr/Lenovo-Th%C4%B1nkpad-21JT0017TX-RYZEN7-7730U-Notebook/dp/B0DBM6N3FT/ref=sr_1_165?dib=eyJ2IjoiMSJ9.PUpYjARowaenHti_yJ9sLr42gcQj-cxOj4aUBUkybZvKfat8sSvnhbDGik4RBSlqNR025uqUJqRdtkZAyT6cHc6yBkpB53hxuf51-sagzwQNXfO_qhkZwn5VtsIuI4z5JcTWjhiRp4EHFux3-lPz06jACGByVD6xSQqX75_cDvBoPSVFJXWHSHhw7Y0jav1uXJ2mPw0hynLmhI9uV2pxplJTzbhFHEmQcFuxPMbZx1WYkvnAgTee87AaewEkBl46ImGv47ZRPcBS-zrgxLiRNbKIaM_-HTwI0HsaW8AS63x6pPpiNWtBONs6S1bg1T-22K_UtE90sveraCdh9sul_BowzfL6SehfGkY2d0OsvHc.iV51EoR2MFjKxI5XLUaFhK5t2V6ZmbwsL0UgEg9ttsk&dib_tag=se&qid=1738968027&s=computers&sr=1-165

Ürün: Asus Notebook Laptop, F1502ZA-EJ1531 FHD, i3-1215U İşlemci, 8 GB RAM, 256 GB SSD, Share, Dos
Link: https://www.amazon.com.tr/Notebook-Laptop-F1502ZA-EJ1531-i3-1215U-%C4%B0%C5%9Flemci/dp/B0CJYF4XT8/ref=sr_1_181?dib=eyJ2IjoiMSJ9.Z7LY3menxp1_Dm_2Y0IKTxXEhg9jgBIhYmmQ6dyP60sIs2TbxdJKnO9CI-5MQr6Kv1rLdNIYD6yv9ibtT-6vko-jBSZGJHV

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         186 non-null    object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 8.8+ KB


,Product Name,Price,Image URL,Link,ASIN,Review
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...,B0DCYYFMYC,Ürünü bilgisayar öğretmeni arkadaşımın tavsiye...
122,Monster Abra A5 V20.4 Intel Core i5 12450H 16 ...,,https://m.media-amazon.com/images/I/61tOfogRfX...,https://www.amazon.com.tr/Monster-Abra-A5-V20-...,B0CHTGBBHT,Paketleme güzeldi. hızlı ve akıcı bir pc oyunl...
168,Monster Abra A7 V15.1.2 Intel Core i7 12700H 3...,,https://m.media-amazon.com/images/I/61IhJi0-db...,https://www.amazon.com.tr/Monster-Abra-A7-V15-...,B0D2LKXJ51,Teşekkür ederim gayet istediğim gibi geldi söy...
180,Lenovo V15 82TT00A5TX i5-1235U 8GB 512SSD 15.6...,,https://m.media-amazon.com/images/I/613UWfBrlJ...,https://www.amazon.com.tr/82TT00A5TX-i5-1235U-...,B0CDGFGKXM,Ürün dün geldi elime Malesef garantisi 2023 ar...


In [15]:
df["Link"][180]

'https://www.amazon.com.tr/82TT00A5TX-i5-1235U-FreeDOS-Diz%C3%BCst%C3%BC-Bilgisayar/dp/B0CDGFGKXM/ref=sr_1_339?dib=eyJ2IjoiMSJ9.S1JcaV1XK1ub2CL8xGkxBjEyGYockkGnuzYvY7VzOXX5pwn50YTZvQx7PRQLOY4lybR5YwTWXoj_t5lI6b1-lyhVc-qxKmc54qn4UYC6AvxmAZQ-Apeoy2uzX5JowKM0jKfdOLMZBlp2wCyU4ENhgTwhbg0_ogtUH5awzjyRhJRM8oPcnJAtFSA-vNn6u5i71l-q4y9panCPP3V9gyIYbwUnccAMMGNbdhoqssNc7iOhAhAm7x2gmvp6T_h5vH_9CPSS4S4xWWNwhsEi4nqPPQ8dfnK_w8b1Pqe3HaKlzPOQ_tkBqfD7txzgHMa1yVvIUzafVfr7RYe1E2T9_wPl3y0SQJQw9wYemkTmoL5zxgg.KgJMNASpw_07-s39j8RzwHVdkRVxWqMXbqRiDb8W4zE&dib_tag=se&qid=1738968207&s=computers&sr=1-339'

In [21]:
df.head(10)

,Product Name,Price,Image URL,Link,ASIN,Review
0,Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...,15.499 TL,https://m.media-amazon.com/images/I/61wH5UlX-M...,https://www.amazon.com.tr/Lenovo-Bilgisayar-i5...,B0CK84VL9X,"Harika bir bilgisayar, gönderim çok hızlı soru..."
1,ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...,14.999 TL,https://m.media-amazon.com/images/I/51dHk31mrQ...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...,B08H1WX5MY,"Günlük kullanıma uygun, şık görünümlü ve perfo..."
2,"Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...",11.659 TL,https://m.media-amazon.com/images/I/61DQ67Cr2H...,https://www.amazon.com.tr/Lenovo-IdeaPad-Bilgi...,B0CQTKWH2M,Akademik işlerim için bu ürünü aldım. Fiyat ve...
3,ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...,10.999 TL,https://m.media-amazon.com/images/I/51zyo-VeNS...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...,B08HLQTYXW,Yıllarca zenbook serisini kullanmış birisi ola...
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,None,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...,B0DCYYFMYC,Ürünü bilgisayar öğretmeni arkadaşımın tavsiye...
5,"MSI CYBORG 15 A13VF-894XTR Dizüstü Bilgisayar,...",45.999 TL,https://m.media-amazon.com/images/I/71l4M2y8zw...,https://www.amazon.com.tr/MSI-A13VF-894XTR-Diz...,B0CZ31GM8L,düşünmeden alın. tam bi fp ürünü diyebilirim. ...
6,"HP 15 Dizüstü Bilgisayar, 15,6 inç FHD, Intel ...",46.897 TL,https://m.media-amazon.com/images/I/812XsG-QIr...,https://www.amazon.com.tr/HP-Diz%C3%BCst%C3%BC...,B0CQRKSDYC,Bu paraya alınacak en mantıklı pc. windows yük...
7,Apple M2 çipli MacBook Air laptop 13.6 inç Liq...,40.399 TL,https://m.media-amazon.com/images/I/719C6bJv8j...,https://www.amazon.com.tr/Apple-MacBook-depola...,B0DLHMLFZL,kargo başarılıydı. Pc için diyecek laf yok zat...
8,Apple M3 çipli 13 inç MacBook Air Laptop: 13.6...,10.899 TL,https://m.media-amazon.com/images/I/71Yu0FBKPi...,https://www.amazon.com.tr/Apple-%C3%A7ipli-in%...,B0DLHDC186,Çeşitli marka laptop kullanmış biri olarak Mac...
9,"Apple MacBook Pro Laptop, M4 Pro çip, 12 çekir...",84.204 TL,https://m.media-amazon.com/images/I/618r76w5dG...,https://www.amazon.com.tr/Apple-MacBook-Laptop...,B0DLHCCM2J,Amazonun paketlemesi inanılmaz güvenli. Apple ...


In [20]:
def format_price(price):
    if isinstance(price, str) and "TL" in price:
        price = price.replace(" TL", "")  # " TL" kısmını kaldır
        try:
            price = float(price)  # Sayıya çevir
            return f"{price:.3f} TL"  # Noktadan sonra 3 basamak bırak ve TL ekle
        except ValueError:
            return None
    return None

df["Price"] = df["Price"].apply(format_price)

In [22]:
df.to_csv("C://Users/Emrehan/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv", index=False)